In [38]:
import pandas as pd
import math
import numpy as np
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gdp

In [39]:
fips_df = pd.read_csv('data/fips2county.tsv', sep='\t', header='infer', dtype=str, encoding='latin-1')
cancer_df = pd.read_csv('data/cancer_reg.csv', encoding='latin-1')

In [40]:
# add a new column 'Target_div_Income'
cancer_df['Target_div_Income'] = cancer_df['TARGET_deathRate'] / cancer_df['medIncome']

# extract state and county from 'Geography' column and create new columns
cancer_df[['County', 'State']] = cancer_df['Geography'].str.extract(r'(.+), (.+)')
cancer_df['County'] = cancer_df['County'].str.replace(' County', '')

# manually change two county names
cancer_df.loc[166, 'County'] = 'Dona Ana County'
cancer_df.loc[820, 'County'] = 'La Salle Parish'

# merge the dataframes to get the FIPS codes
cancer_df = pd.merge(cancer_df, fips_df,
                     left_on=['County'], right_on=['CountyName'], how='left')

# add a new column 'Target_div_LogIncome'
cancer_df['Target_div_LogIncome'] = cancer_df['TARGET_deathRate'] / \
                                    (cancer_df['medIncome'].apply(lambda x: math.log(x)))

cancer_df
# create a new dataframe for graphdata with columns 'fips' and 'values'
graphdata = pd.DataFrame({'fips': cancer_df['CountyFIPS'],
                          'values': cancer_df['Target_div_LogIncome'],
                          'CountyFIPS': cancer_df['CountyFIPS']})


In [45]:
# create a new dataframe newbieLOG with column 'anomalies'
newbieLOG = graphdata.copy()
newbieLOG['anomalies'] = (newbieLOG['values'] - newbieLOG['values'].mean()) / newbieLOG['values'].std()
newbieLOG['anomalies'] = np.where((newbieLOG['anomalies']) > 1, (newbieLOG['anomalies']), 0)
newbieLOG = newbieLOG[['fips', 'anomalies']]
newbieLOG

,fips,anomalies
0,53035,0.000000
1,53037,0.000000
2,53039,0.000000
3,16061,0.000000
4,21135,0.000000
...,...,...
14223,48159,0.000000
14224,50011,0.000000
14225,51067,0.000000
14226,53021,0.000000
